# Package

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import shap
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr
from sklearn.utils import shuffle
import pickle

d:\Portofolio Data science\Time Series\Travaux pratiques Bases\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Importation

In [ ]:
# Les données
df_stationary_test = pd.read_csv("df_stationary_test.csv", index_col="date")
df_stationary_test.index = pd.to_datetime(df_stationary_test.index)

In [5]:
import pickle

# Nom du fichier du modèle sauvegardé
file_name = "OLS_h12_expanding_window.pkl"

# 🔁 Charger le fichier
with open(file_name, "rb") as f:
    exp_results = pickle.load(f)

# Extraire uniquement les modèles
models = exp_results["models"]

print(f"✅ {len(models)} modèles OLS rechargés avec succès depuis '{file_name}'.")

✅ 29 modèles OLS rechargés avec succès depuis 'OLS_h12_expanding_window.pkl'.


# 1) Paramètres & sous-ensemble TRAIN global

In [7]:
# ================== TEST AVEC MODÈLE PRÉ-ENTRAÎNÉ (reconstruit les transfos) ==================
import pickle
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

# 1) Charger le pack
with open("OLS_h12_expanding_window.pkl", "rb") as f:
    exp_results = pickle.load(f)

models        = exp_results["models"]
train_periods = exp_results["train_periods"]
params        = exp_results["params"]

# Récup params (fallback si manquants)
h             = params.get("h", 12)
winsor_level  = params.get("winsor_level", 0.01)
cols_tx       = params.get("features", [c for c in df_stationary_train.columns if c != "UNRATE"])

# 2) RECONSTRUIRE les transfos de la DERNIÈRE fenêtre d'entraînement
last_train_end = pd.to_datetime(train_periods[-1])  # ex. 1989-12
df_train_global = df_stationary_train.loc[:last_train_end].copy().sort_index()

# Aligner X_t avec Y_{t+h} pour le fit (comme lors de l'entraînement)
if len(df_train_global) <= h:
    raise ValueError("Fenêtre d'entraînement trop courte pour reconstruire les transfos.")
X_last = df_train_global.loc[:, cols_tx].iloc[:-h].copy()
Y_last = df_train_global["UNRATE"].shift(-h).iloc[:-h].copy()
valid  = ~(X_last.isnull().any(axis=1) | Y_last.isnull())
X_last = X_last.loc[valid]

# Winsor + normalisation (transfos à réutiliser sur le test)
lower_wins = X_last.quantile(winsor_level)
upper_wins = X_last.quantile(1 - winsor_level)
Xw_last    = X_last.clip(lower=lower_wins, upper=upper_wins, axis=1)
mean_last  = Xw_last.mean()
std_last   = Xw_last.std().replace(0, 1)

# 3) Jeu de TEST (dates postérieures à last_train_end)
df_test_full = df_stationary_test.copy().sort_index()
df_test      = df_test_full.loc[last_train_end:]  # pour chaque t on prédit y_{t+h}

# 4) Prédire à horizon h avec le DERNIER modèle (aucune réestimation)
model_final = models[-1]
records = []

for t_date in df_test.index:
    # vérifier t+h dans df_test_full
    try:
        t_pos = df_test_full.index.get_loc(t_date)
    except KeyError:
        continue
    target_pos = t_pos + h
    if target_pos >= len(df_test_full):
        break

    # X_t (1xK) + mêmes transfos que sur la dernière fenêtre d'entraînement
    x_t  = df_test_full.loc[[t_date], cols_tx].copy()
    xw_t = x_t.clip(lower=lower_wins, upper=upper_wins, axis=1)
    xs_t = (xw_t - mean_last) / std_last

    y_hat = float(model_final.predict(xs_t)[0])
    target_date = df_test_full.index[target_pos]
    y_true = float(df_test_full.loc[target_date, "UNRATE"])

    records.append({"origin": t_date, "target": target_date, "y_hat": y_hat, "y_true": y_true})

forecast_test_df = pd.DataFrame(records)

# 5) Évaluation
if forecast_test_df.empty:
    print("Aucune prévision produite sur le test (vérifie dates et horizon h).")
else:
    y, yhat = forecast_test_df["y_true"].values, forecast_test_df["y_hat"].values
    r2   = r2_score(y, yhat)
    mae  = mean_absolute_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))
    corr = np.corrcoef(y, yhat)[0, 1]

    print(f"\n=== TEST pseudo-OOS (h = {h} mois) ===")
    print(f"R²   : {r2:.3f}\nMAE  : {mae:.3f}\nRMSE : {rmse:.3f}\nCorr : {corr:.3f}")

    forecast_test_df.to_csv("forecast_test_OLS.csv", index=False)
    print("Prévisions test exportées -> forecast_test_OLS.csv")

NameError: name 'df_stationary_train' is not defined

In [21]:
# ===================== ÉVALUATION PSEUDO–OUT-OF-SAMPLE =====================
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr

# 1️⃣ Rassembler les prévisions
forecast_df = pd.DataFrame(forecast_records).dropna(subset=["y_true", "y_hat"]).copy()
forecast_df = forecast_df.sort_values("target").reset_index(drop=True)

def r2_origin_reg(y, yhat):
    """R² d'une régression à l’origine: y ≈ b * yhat (sans intercept)."""
    denom = np.dot(yhat, yhat)
    if denom == 0:
        return np.nan
    b = np.dot(yhat, y) / denom
    sse = np.sum((y - b * yhat) ** 2)
    sst = np.sum((y - y.mean()) ** 2)
    return 1 - sse / sst if sst > 0 else np.nan

def corr_pvalue(y, yhat):
    if len(y) < 3:
        return np.nan, np.nan
    r, p = pearsonr(y, yhat)
    return float(r), float(p)

if forecast_df.empty:
    print("\n[ÉVALUATION] Aucune prévision disponible (forecast_df est vide).")
else:
    y = forecast_df["y_true"].values
    yhat = forecast_df["y_hat"].values

    # 2️⃣ Métriques principales
    r2   = r2_score(y, yhat)
    mae  = mean_absolute_error(y, yhat)
    rmse = np.sqrt(mean_squared_error(y, yhat))   # ✅ compatible toutes versions
    r2_o = r2_origin_reg(y, yhat)
    corr, pval = corr_pvalue(y, yhat)
    amd  = float(abs(np.mean(y - yhat)))          # biais absolu moyen

    # 3️⃣ Benchmark naïf (utile si la cible est Δ12 UNRATE)
    yhat_naive0 = np.zeros_like(y)
    mae_naive0  = mean_absolute_error(y, yhat_naive0)
    rmse_naive0 = np.sqrt(mean_squared_error(y, yhat_naive0))

    # 4️⃣ Précision directionnelle
    hit_rate = float(np.mean(np.sign(y) == np.sign(yhat))) if len(y) > 0 else np.nan

    # 5️⃣ Calibration (Mincer–Zarnowitz): y = a + b * yhat
    reg = LinearRegression()
    reg.fit(yhat.reshape(-1, 1), y)
    a_calib = float(reg.intercept_)
    b_calib = float(reg.coef_[0])

    # 6️⃣ Résumé par décennie (facultatif)
    by_decade = (
        forecast_df.assign(decade=forecast_df["target"].dt.year // 10 * 10)
                   .groupby("decade")
                   .apply(lambda g: pd.Series({
                       "n": len(g),
                       "MAE": mean_absolute_error(g["y_true"], g["y_hat"]),
                       "RMSE": np.sqrt(mean_squared_error(g["y_true"], g["y_hat"]))
                   }))
                   .reset_index()
    )

    # 7️⃣ Impression des résultats
    print("\n=== ÉVALUATION PSEUDO–OOS (h = {} mois) ===".format(h))
    print(f"R²            : {r2:.3f}")
    print(f"R² (origin)   : {r2_o:.3f}")
    print(f"MAE           : {mae:.3f}")
    print(f"RMSE          : {rmse:.3f}")
    print(f"Corr(y, ŷ)   : {corr:.3f}  (p={pval:.3g})")
    print(f"Hit rate sign : {hit_rate:.3f}")
    print(f"AMD (|bias|)  : {amd:.3f}")

    print("\n--- Benchmark naïf (zéro-changement) ---")
    print(f"MAE_naïf0     : {mae_naive0:.3f}")
    print(f"RMSE_naïf0    : {rmse_naive0:.3f}")

    if not by_decade.empty:
        print("\n--- MAE/RMSE par décennie ---")
        print(by_decade.to_string(index=False))

    # 8️⃣ Sauvegarde des résultats
    eval_results = {
        "overall": {
            "r2": r2, "r2_origin": r2_o, "mae": mae, "rmse": rmse,
            "corr": corr, "pval": pval, "hit_rate": hit_rate, "amd": amd
        },
        "benchmark_naive0": {
            "mae": mae_naive0, "rmse": rmse_naive0
        },
        "calibration": {
            "intercept": a_calib, "slope": b_calib
        },
        "by_decade": by_decade,
        "forecast_df": forecast_df
    }

    print("\n✅ Évaluation terminée. Résultats enregistrés dans eval_results.")


=== ÉVALUATION PSEUDO–OOS (h = 12 mois) ===
R²            : 0.095
R² (origin)   : 0.205
MAE           : 0.809
RMSE          : 1.058
Corr(y, ŷ)   : 0.453  (p=0.0154)
Hit rate sign : 0.714
AMD (|bias|)  : 0.010

--- Benchmark naïf (zéro-changement) ---
MAE_naïf0     : 0.800
RMSE_naïf0    : 1.113

--- MAE/RMSE par décennie ---
 decade    n      MAE     RMSE
   1960  8.0 0.695125 1.136080
   1970 10.0 0.772075 0.930614
   1980 10.0 0.937675 1.112530

✅ Évaluation terminée. Résultats enregistrés dans eval_results.


C:\Users\Mita\AppData\Local\Temp\ipykernel_14108\1481986021.py:60: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: pd.Series({


In [ ]:
# ==========================================================
# 🔍 IMPORTANCE PAR PERMUTATION — PSEUDO-OOS
# ==========================================================
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error

def permutation_importance_pseudo_oos(models, df_train_global, cols_tx, h=12, n_repeats=20, metric=mean_absolute_error):
    """
    Importance par permutation pour une série de modèles OLS entraînés
    selon une logique expanding window pseudo–OOS.
    -> aucune réestimation
    -> mesure la dégradation moyenne de la performance après permutation de chaque variable
    """
    var_imp = {col: [] for col in cols_tx}

    for i, model in enumerate(models):
        # Reconstituer la fenêtre utilisée par le modèle i
        end_idx = (i + 1) * 12  # correspond à ton step_size = 12
        df_win = df_train_global.iloc[:end_idx].copy()

        if len(df_win) <= h:
            continue

        # Préparation des données (alignement X_t avec Y_{t+h})
        X = df_win[cols_tx].iloc[:-h].copy()
        y = df_win["UNRATE"].shift(-h).iloc[:-h].copy()
        valid = ~(X.isnull().any(axis=1) | y.isnull())
        X, y = X.loc[valid], y.loc[valid]

        # Score de base (MAE sur données d'origine)
        base_score = metric(y, model.predict(X))

        # Boucle sur chaque variable
        for col in cols_tx:
            perm_scores = []
            for _ in range(n_repeats):
                X_perm = X.copy()
                X_perm[col] = np.random.permutation(X_perm[col])
                perm_scores.append(metric(y, model.predict(X_perm)))
            perm_scores = np.array(perm_scores)
            var_imp[col].append(np.mean(perm_scores) / base_score)

    # Agrégation moyenne sur toutes les fenêtres
    results = []
    for col, ratios in var_imp.items():
        if len(ratios) > 0:
            results.append({
                "variable": col,
                "perm_mae_ratio_mean": np.mean(ratios),
                "perm_mae_ratio_std": np.std(ratios),
                "n_windows": len(ratios)
            })

    imp_df = pd.DataFrame(results).sort_values("perm_mae_ratio_mean", ascending=False).reset_index(drop=True)
    return imp_df

In [ ]:
# ----------------------------------------------------------
# 🧭 Appel de la fonction sur ton jeu de modèles OLS
# ----------------------------------------------------------
perm_df = permutation_importance_pseudo_oos(
    models=models,
    df_train_global=df_train_global,
    cols_tx=cols_tx,
    h=h,
    n_repeats=20  # augmente à 50 pour des résultats plus stables
)

print("\n=== 🔍 Importance par permutation (pseudo–OOS) ===")
print(perm_df.head(15).to_string(index=False))


=== 🔍 Importance par permutation (pseudo–OOS) ===
       variable  perm_mae_ratio_mean  perm_mae_ratio_std  n_windows
          USREC             1.092658            0.051073         28
          TB3MS             1.028961            0.025485         28
        S&P 500             1.013475            0.004537         28
      OILPRICEx             1.004680            0.004453         28
DPCERA3M086SBEA             1.000228            0.000298         28
           M2SL             1.000208            0.000311         28
         INDPRO             1.000093            0.001816         28
       BUSLOANS             1.000056            0.000236         28
            RPI             1.000051            0.000412         28
       CPIAUCSL             1.000038            0.000150         28


Ton modèle OLS prédit principalement le chômage via le cycle économique :
- USREC (récession) est la variable clé — sa permutation dégrade la performance de ~9 %.
- TB3MS (taux court) joue un rôle secondaire.
- Les autres variables ont un effet négligeable.

Conclusion : le pouvoir prédictif du modèle vient surtout des variables cycliques (récession, taux), les autres apportent peu d’information.

In [ ]:
# ==========================================================
# 💡 IMPORTANCE SHAPLEY (pour modèle OLS)
# ==========================================================
import shap
import numpy as np
import pandas as pd

# 1️⃣ On utilise le dernier modèle entraîné
model_final = models[-1]

# 2️⃣ On reconstitue ses données finales (dernière fenêtre du train)
df_final = df_train_global.copy()
X_full = df_final[cols_tx].iloc[:-h].copy()
Y_full = df_final["UNRATE"].shift(-h).iloc[:-h].copy()
valid = ~(X_full.isnull().any(axis=1) | Y_full.isnull())
X_full = X_full.loc[valid]
Y_full = Y_full.loc[valid]

# 3️⃣ Calcul des valeurs SHAP
# Pour les modèles linéaires, on peut utiliser shap.LinearExplainer (plus stable)
explainer = shap.LinearExplainer(model_final, X_full, feature_perturbation="interventional")
shap_values = explainer(X_full)

# 4️⃣ Importance moyenne absolue
shap_df = pd.DataFrame({
    "variable": X_full.columns,
    "shap_mean_abs": np.abs(shap_values.values).mean(axis=0),
})
shap_df["shap_share"] = shap_df["shap_mean_abs"] / shap_df["shap_mean_abs"].sum()
shap_df = shap_df.sort_values("shap_mean_abs", ascending=False).reset_index(drop=True)

# 5️⃣ Affichage
print("\n=== 💡 Importance SHAP (shares) ===")
print(shap_df.head(10).to_string(index=False))


=== 💡 Importance SHAP (shares) ===
       variable  shap_mean_abs  shap_share
          TB3MS       0.148710    0.485751
          USREC       0.124740    0.407453
        S&P 500       0.017688    0.057777
      OILPRICEx       0.012534    0.040941
DPCERA3M086SBEA       0.001011    0.003303
         INDPRO       0.000871    0.002845
       CPIAUCSL       0.000309    0.001010
           M2SL       0.000204    0.000667
            RPI       0.000043    0.000139
       BUSLOANS       0.000035    0.000114


d:\Portofolio Data science\Time Series\Travaux pratiques Bases\.venv\Lib\site-packages\shap\explainers\_linear.py:99: FutureWarning: The feature_perturbation option is now deprecated in favor of using the appropriate masker (maskers.Independent, maskers.Partition or maskers.Impute).
  warnings.warn(wmsg, FutureWarning)


- TB3MS (0.49)	🟢 Représente ~49 % de l’influence totale du modèle. Le taux d’intérêt à 3 mois est donc la variable la plus déterminante pour les prévisions du chômage : quand les taux montent, le modèle anticipe souvent une hausse future du chômage.

- USREC (0.41)	🔵 Représente ~41 % de l’influence totale. Le dummy de récession (NBER) pèse presque autant : le simple fait d’être en récession ou non explique une large part des variations prévues du chômage.

- S&P 500, OILPRICEx 🟠 Poids faibles (~6 % et 4 %) : les conditions boursières et le prix du pétrole ont un impact marginal dans la version linéaire du modèle.

# 📊 Métriques utilisées

## 1) Performance globale

**Coefficient de détermination (R²)**  
$$
R^2 = 1 - \frac{\sum_i (y_i - \hat{y}_i)^2}{\sum_i (y_i - \bar{y})^2}
$$  
➡️ Part de la variance expliquée par le modèle (0 = pas mieux que la moyenne, 1 = parfait).

**Erreur absolue moyenne (MAE)**  
$$
MAE = \frac{1}{n}\sum_{i=1}^n |y_i - \hat{y}_i|
$$  
➡️ Écart absolu moyen entre valeurs réelles et prédites, robuste aux outliers.

**Erreur quadratique moyenne (RMSE)**  
$$
RMSE = \sqrt{\frac{1}{n}\sum_{i=1}^n (y_i - \hat{y}_i)^2}
$$  
➡️ Similaire au MAE mais pénalise davantage les grosses erreurs.

**Corrélation de Pearson**  
$$
\rho(y, \hat{y}) = \frac{\text{Cov}(y, \hat{y})}{\sigma_y \cdot \sigma_{\hat{y}}}
$$  
➡️ Mesure le degré de lien linéaire entre les prédictions et les observations.

**Abs Mean Deviance (AMD)**  
$$
AMD = \frac{1}{n}\sum_{i=1}^n |\hat{y}_i - \bar{\hat{y}}|
$$  
➡️ Écart moyen des prédictions par rapport à leur moyenne ; sert de référence pour la permutation prédiction-basée.

---

## 2) Importance par permutation
La relation entre Y et X dépend du temps. Quand on perturbe la série X (en la mélangeant), on casse ce lien, et si l’erreur augmente, cela montre que X est une variable clé pour expliquer Y.

**Ratio MAE**  
$$
PI^{MAE}_j = \frac{MAE^{(perm)}_j}{MAE^{(base)}}
$$  
➡️ Si > 1, la variable est utile pour réduire l’erreur absolue.

**Ratio RMSE**  
$$
PI^{RMSE}_j = \frac{RMSE^{(perm)}_j}{RMSE^{(base)}}
$$  
➡️ Si > 1, la variable aide à limiter les grosses erreurs.

**Déviance de prédiction**  
$$
PI^{dev}_j = \frac{1}{n}\sum_{i=1}^n \big|\hat{y}_i - \hat{y}^{(perm)}_{i,j}\big|
$$  
➡️ Mesure combien les prédictions changent quand on brouille une variable.

---

## 3) Importance Shapley

**Décomposition des prédictions**  
$$
\hat{y}_i = \phi_0 + \sum_{j=1}^p \phi_{ij}
$$  
➡️ Chaque prédiction est expliquée par une contribution \(\phi_{ij}\) par variable.

**Importance absolue moyenne**  
$$
\text{Mean}(|\phi_j|) = \frac{1}{n}\sum_{i=1}^n |\phi_{ij}|
$$  
➡️ Contribution moyenne (absolue) d’une variable sur toutes les prédictions.

**Shapley share**  
$$
\Gamma_j = \frac{\text{Mean}(|\phi_j|)}{\sum_{k=1}^p \text{Mean}(|\phi_k|)}
$$  
➡️ Part relative de la variable dans l’explication totale (somme des parts = 1).

## Interprétation des résultats 

### Performance globale 
- R² = 0.2266 → modèle OLS explique ~23 % de la variance du chômage US.
- MAE = 0.6774 → en moyenne, l’erreur absolue est de 0.68 points (dans l’unité de la variable cible).
- RMSE = 0.8750 → un peu plus élevé que le MAE, ce qui indique la présence de grosses erreurs ponctuelles.
- Corrélation = 0.4760 (p ≈ 10⁻³⁵) → lien positif et significatif entre prédictions et observations, mais seulement modéré. Ce qui peut expliquer la présence d'une relation 
- Abs Mean Deviance = 0.3370 → sert ici de référence pour l’importance prédiction-basée : les prédictions s’écartent en moyenne de 0.34 de leur propre moyenne.

Lecture : le modèle OLS capte une partie utile du signal, mais laisse beaucoup de variance inexpliquée. La corrélation faible illustre éventuellement la présence des relations non-linéaire, et non captées par OLS.

### 🔹 2. Importance par permutation
- INDPRO (Industrial Production) : la plus influente. Sa permutation augmente MAE de +19 % et RMSE de +20 %, avec une forte déviance de prédiction (0.41).
- TB3MS (Taux d’intérêt à 3 mois) : impact non négligeable, ratios ~1.02 et déviance ~0.10.
- BUSLOANS (Prêts commerciaux) : rôle similaire (MAE ratio 1.017, déviance ~0.09).
- S&P 500 : contribution modérée, ratios légèrement > 1.
- RPI, M2SL : influence plus faible mais perceptible.
- CPIAUCSL, OILPRICEx, DPCERA3M086SBEA : quasi neutres (ratios ≈ 1, déviance très faible).

Lecture : INDPRO domine largement la performance, les autres apportent des compléments mais plus modestes.

### 🔹 3. Importance Shapley (shares)
- INDPRO : ~52 % de l’explication totale des prédictions → cohérence parfaite avec la permutation.
- TB3MS (12 %) + BUSLOANS (12 %) : deux autres piliers importants.
- S&P 500 (9,7 %) : contribue de façon notable.
- M2SL (5 %), RPI (3 %), CPIAUCSL (3,5 %) : apports plus secondaires.
- OILPRICEx et DPCERA3M086SBEA (<2 %) : quasi négligeables dans ce modèle.

Lecture : INDPRO est la variable macroéconomique centrale, suivie par des indicateurs financiers (taux courts, prêts bancaires, marché actions).

# Test

In [ ]:
import joblib

saved_model = joblib.load("models/model_final.pkl")
model_final = saved_model["model"]
features = saved_model["features"]
winsor_level = saved_model["winsor_level"]
norm_var = saved_model["norm_var"]
mean_full = saved_model["mean_full"]
std_full = saved_model["std_full"]